In [202]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas
import seaborn as sns
from tqdm import tqdm

sns.set(rc={'figure.figsize':(10,10)})

In [203]:
os.chdir('/Users/sreelakshmirajendrakumar/Downloads')

dataset_pendigits_test = pandas.read_csv('pendigits_test.csv')
dataset_pendigits_train = pandas.read_csv('pendigits_train.csv')
dataset_class_2D = pandas.read_csv('class_2D.csv')

In [204]:
X_train=dataset_pendigits_train.iloc[:,0:16]
X_train=X_train.to_numpy()
N,d=np.shape(X_train)

y_train=dataset_pendigits_train.iloc[:,16]
y_train=y_train.to_numpy()
y_train=np.array(y_train).astype(int)

X_test=dataset_pendigits_test.iloc[:,0:16]
X_test=X_test.to_numpy()

y_test=dataset_pendigits_test.iloc[:,16]
y_test=y_test.to_numpy()
y_test=np.array(y_test).astype(int)

gamma_zero=10

In [205]:
X_train_sorted=[]
for i in range(0,10):
    X_train_sorted.append([]) #(10,) (589,16)
    for j in range(0,len(X_train)):
        if y_train[j]==i:
            X_train_sorted[i].append(X_train[j])

In [206]:
def generateDiagonalY(y_,N):
    'creating a matrix with diagonal values equal to the elements in y'
    diagonal_y=np.zeros((N,N))
    for i in range(0,len(diagonal_y)):
        for j in range(0,len(diagonal_y)):
            if i==j:
                diagonal_y[i][j]=y_[i]
    return diagonal_y

def changeY(to_change_y,c):
    '''changing y such that the samples belonging to the class under consideration has y=1 and others have y=-1'''
    changed_y=np.zeros(len(to_change_y),)
    for i in range(0,len(to_change_y)):
        if to_change_y[i]!=c:
            changed_y[i]=-1
        elif to_change_y[i]==c:
            changed_y[i]=1
    return changed_y


changed_y=[] # (10,589)
for i in range(0,10):
    changed_y.append(changeY(y_train,i))

diagonal_y=[] # (10,589,589) 
for i in range(0,10):
    diagonal_y.append([])
    diagonal_y[i]=generateDiagonalY(changed_y[i],N)

In [207]:
def numPos(array):
    w=0
    for i in range(0,len(array)):
        if array[i]>0:
            w+=1
    return w

'''getting the number of elements belonging to each class'''
N_class=[] # (10,)
for i in range(0,10):
    N_class.append(numPos(changed_y[i]))

In [208]:
def attachOnes(X=X_train):
    'attaching ones to the first column of the input matrix X'
    to_attach=np.zeros((len(X),1))
    to_attach+=1
    attached=np.concatenate((to_attach,X),axis=1)
    return attached

attached_x=attachOnes()
A=[]
W=[]
for i in range(0,10):
    A.append(np.matmul(diagonal_y[i],attached_x)) #(10, 589, 17)
    W.append(np.matmul(diagonal_y[i],X_train)) # (10, 589, 16)
    

In [209]:
'''getting gamma'''
gamma=np.zeros(len(y_train),)
for j in range(0,len(y_train)):
    for i in range(0,10):
        if y_train[j]==i:
            temp=10*len(y_train)/N_class[i]
            gamma[j]=temp

In [210]:
def getU(the_z,gamma=gamma):
    u=np.zeros(len(gamma),)
    for i in range(0,len(u)):
        if the_z[i]<1:
            u[i]=(-1)*gamma[i]
    return u

In [246]:
initial_beta_zero=[]
initial_b=[]
initial_z=[]
initial_u=[]

temp_beta_zero=[]
temp_b=[]
temp_z=[]
temp_u=[]

current_beta_zero=[] # (10,)
current_b=[] # (10,16)
current_z=[] # (10,589)
current_u=[] # (10,589)

rho=0.02
tolerance=10**(-5)

for i in range(0,10):
    '''initializing the values'''
    initial_beta_zero.append(np.random.rand(1,))
    initial_b.append(np.random.rand(d,))
    initial_z.append(np.random.rand(N,))
    initial_u.append(np.random.rand(N,))

    temp_beta_zero.append(np.zeros(1,))
    temp_b.append(np.zeros((N,d)))
    temp_z.append(np.zeros(N,))
    temp_u.append(np.zeros(N,))

    current_beta_zero.append(initial_beta_zero[i])
    current_b.append(initial_b[i])
    current_z.append(initial_z[i])
    current_u.append(initial_u[i])
    
    

    
for i in tqdm(range(0,10)):
    temp1=0;temp2=0;temp3=0;temp4=0;temp5=0;temp6=0;temp7=0
    for j in range(0,10000):
        temp_beta_zero[i]=current_beta_zero[i]
        temp1=current_z[i]-current_u[i]-np.matmul(W[i],current_b[i])
        current_beta_zero[i]=(np.matmul(changed_y[i].T,temp1)/(np.matmul(changed_y[i].T,changed_y[i])))
    
        temp_b[i]=current_b[i]
        temp2=rho*np.identity(d)+np.matmul(W[i].T,W[i])
        temp3=np.linalg.inv(temp2)
        temp4=current_z[i]-current_u[i]-current_beta_zero[i]*changed_y[i]
        temp5=np.matmul(temp3,W[i].T)
        current_b[i]=np.matmul(temp5,temp4)
    
        temp_z[i]=current_z[i]
        temp6=getU(temp_z[i],gamma)
        current_z[i]=-rho*temp6+(changed_y[i]*current_beta_zero[i])+np.matmul(W[i],current_b[i])+current_u[i]
    
        temp_u[i]=current_u[i]
        temp7=temp_u[i]+(changed_y[i]*current_beta_zero[i])+np.matmul(W[i],current_b[i])-current_z[i]
        current_u[i]=temp7
    
#     if abs(temp_beta_zero[i]-current_beta_zero[i])<tolerance:
#         temp8=abs(temp_b[i]-current_b[i])
#         if np.all(temp8<tolerance):
#             break


100%|██████████| 10/10 [00:43<00:00,  4.37s/it]


In [267]:
def getScore(beta_zero,b,X):
    for i in range(0,len(X)):
        c=beta_zero+np.matmul(X.T,b)
    return c

In [268]:
assigned_classes=[]
for j in range(0,len(X_test)):
    temp_array=[]
    for i in range(0,10):
        temp_value=getScore(current_beta_zero[i],current_b[i],X_test[j])
        temp_array.append(temp_value)
    assigned_classes.append(np.argmax(temp_array))    
    
    

In [276]:
import sklearn
from sklearn.metrics import confusion_matrix

sklearn.metrics.confusion_matrix(assigned_classes,y_test, labels=None, sample_weight=None)

array([[70,  0,  0,  0,  2,  0,  0,  0,  2,  2],
       [ 0, 81,  2,  1,  0,  0,  0,  1,  1,  5],
       [ 0,  1, 87,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  1, 81,  0,  0,  0,  0,  0,  2],
       [ 2,  1,  0,  0, 97,  0,  2,  0,  0,  1],
       [ 1,  2,  1,  0,  0, 78,  0,  1,  4, 11],
       [ 0,  0,  0,  0,  0,  0, 68,  1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 80,  0,  1],
       [ 3,  0,  0,  0,  0,  2,  1,  0, 63,  0],
       [ 0,  0,  0,  0,  0,  2,  0,  0,  0, 73]])

In [278]:
changed_test_y=[] # (10,589)
for i in range(0,10):
    changed_test_y.append(changeY(y_test,i))
    
N_test_class=[] # (10,)
for i in range(0,10):
    N_test_class.append(numPos(changed_test_y[i]))

In [279]:
N_test_class

[76, 86, 91, 82, 99, 82, 71, 83, 70, 95]